# **Movie Recommendation System**



Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

The difflib library in Python is used for comparing sequences, finding differences, and generating similarity ratios. It helps in text comparison, file difference analysis, and approximate string matching.

If the user makes a typing mistake in the movie name, we use the difflib library to find and suggest the closest matching movie.

TfidfVectorizer is used for converting textual data into Numerical values i.e., feature vectors

In recommendation systems, cosine similarity is used to measure the similarity between users or items based on their feature vectors (e.g., ratings, preferences, or interactions). It helps in:



*   Item-based filtering: Recommending items similar to what a user has liked.
*   User-based filtering: Finding users with similar preferences to make recommendations.



Higher cosine similarity means stronger relevance between users or items.

# Data Collection and Pre-processing

In [ ]:
df = pd.read_csv('/content/movies.csv')

In [ ]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
df.shape

(4803, 24)

We will select only relevant features for the recommendation, excluding the rest.

**Feature Selection**

In [ ]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

In [ ]:
df[selected_features]

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,NaN,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,NaN,NaN,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


**Handling Missing Values**

In [ ]:
df.isnull().sum()

,0
index,0
budget,0
genres,28
homepage,3091
id,0
keywords,412
original_language,0
original_title,0
overview,3
popularity,0









1.  Some columns like id, popularity, and revenue have no missing values, while others like homepage (3091 missing) and tagline (844 missing) have significant gaps. Fields like homepage and tagline are inconsistently populated.
2.   Key columns such as genres (28 missing), director (30 missing), and cast (43 missing) have missing values, which could impact analyses related to genres, directors, or actors. Release_date has 1 missing value, potentially affecting time-based analyses.
3. Missing data in keywords (412 missing), overview (3 missing), and runtime (2
missing) could affect text-based analyses or models. High missing values in homepage and tagline may limit their usability but aren’t critical for all analyses. Preprocessing is needed to address these gaps.







In [ ]:
for feature in selected_features:
    df[feature] = df[feature].fillna('')

We have filled the null values with null string for all the features we selected for our recommendation

In [ ]:
df.isnull().sum()

,0
index,0
budget,0
genres,0
homepage,3091
id,0
keywords,0
original_language,0
original_title,0
overview,3
popularity,0


Here, we can observe that the null values count for the selected features are now '0'

In [ ]:
combined_features = df['genres'] + ' ' + df['keywords'] + ' ' + df['tagline'] + ' ' + df['cast'] + ' ' + df['director']

# Combining all the selected features

In [ ]:
combined_features.shape

(4803,)

In [ ]:
combined_features

,0
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...
...,...
4798,Action Crime Thriller united states\u2013mexic...
4799,Comedy Romance A newlywed couple's honeymoon ...
4800,Comedy Drama Romance TV Movie date love at fir...
4801,A New Yorker in Shanghai Daniel Henney Eliza...


In [ ]:
combined_features[0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society Enter the World of Pandora. Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron'

**Converting Textual data into Feature Vectors**

The conversion is necessary to compute cosine similarity, which requires numerical data instead of text.

In [ ]:
vect = TfidfVectorizer()

In [ ]:
feature_vectors = vect.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

In [ ]:
feature_vectors.shape

(4803, 17318)

In [ ]:
print(feature_vectors[0])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 29 stored elements and shape (1, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  (0, 10229)	0.16058685400095302
  (0, 13024)	0.1942362060108871
  (0, 77

**Similarity Score**

In [ ]:
similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
similarity.shape

(4803, 4803)

From the shape, we can infer that each row contains 4,803 scores, representing the comparison of the respective movie with all other movies.

The similarity score is 1 for itself, while the remaining values correspond to the other movies.

**User Interaction**

Ask the user for a movie name. Check if it exists in the dataset. If it does, suggest similar movies based on similarity scores

In [ ]:
movie_name = input("Enter your favorite Movie Name: ")

Enter your favorite Movie Name: bat man


In [ ]:
list_of_all_movies = df['title'].tolist()  # List of all the movies in the dataset

In [ ]:
list_of_all_movies

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

The reason for collecting all the names in the dataset is to find the best match for the movie name given by the user

**Finding the close match**

In [ ]:
close_match = difflib.get_close_matches(movie_name, list_of_all_movies)

In [ ]:
close_match

['Batman', 'Batman', 'Catwoman']

In [ ]:
closest_match = close_match[0]

In [ ]:
index_of_the_movie = df[df['title']==closest_match]['index'].values[0]

In [ ]:
index_of_the_movie

np.int64(1359)

Getting the list of similar movies

In [ ]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [ ]:
similarity_score

[(0, np.float64(0.025315122697371115)),
 (1, np.float64(0.049832930643991497)),
 (2, np.float64(0.013599520029326729)),
 (3, np.float64(0.20438773732168228)),
 (4, np.float64(0.024929726723526918)),
 (5, np.float64(0.11533013884014888)),
 (6, np.float64(0.0)),
 (7, np.float64(0.005521938648290495)),
 (8, np.float64(0.030934935200526422)),
 (9, np.float64(0.08976169961638153)),
 (10, np.float64(0.1124085019852625)),
 (11, np.float64(0.011808001891564553)),
 (12, np.float64(0.03825857693295372)),
 (13, np.float64(0.013263747027865664)),
 (14, np.float64(0.12815026535769383)),
 (15, np.float64(0.010432758242497994)),
 (16, np.float64(0.005235971636780311)),
 (17, np.float64(0.01601669810036477)),
 (18, np.float64(0.041538815515852534)),
 (19, np.float64(0.02968626711977731)),
 (20, np.float64(0.02521005401012206)),
 (21, np.float64(0.004591239318101996)),
 (22, np.float64(0.01792276874908081)),
 (23, np.float64(0.024870438166594427)),
 (24, np.float64(0.041865507456289)),
 (25, np.float64

In [ ]:
# Sorting based on similarity score

sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse=True)

In [ ]:
sorted_similar_movies

[(1359, np.float64(1.0)),
 (428, np.float64(0.4311643836232694)),
 (210, np.float64(0.2573799982085963)),
 (3, np.float64(0.20438773732168228)),
 (119, np.float64(0.1926252875715041)),
 (65, np.float64(0.17755815066113925)),
 (1512, np.float64(0.14705162654306447)),
 (813, np.float64(0.1441412830396247)),
 (2530, np.float64(0.1373732247372919)),
 (1017, np.float64(0.13713281929528848)),
 (473, np.float64(0.13217075714794208)),
 (753, np.float64(0.13216136203404205)),
 (278, np.float64(0.12996260715124028)),
 (14, np.float64(0.12815026535769383)),
 (72, np.float64(0.12266571244563479)),
 (2313, np.float64(0.1183433298876972)),
 (2381, np.float64(0.11752380293327758)),
 (5, np.float64(0.11533013884014888)),
 (2858, np.float64(0.11413652180839842)),
 (4183, np.float64(0.11353876531321921)),
 (30, np.float64(0.11336219425554916)),
 (2655, np.float64(0.11270953916613298)),
 (10, np.float64(0.1124085019852625)),
 (1035, np.float64(0.11197582745207196)),
 (2685, np.float64(0.10980421440315517

**Printing the name of high Similarity Movies based on Index**

In [ ]:
print('Movies suggested for you:\n')

i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title = df[df['index']==index]['title'].values[0]
    if i<=20:
        print(f'{i}: {title}')
        i+=1

Movies suggested for you:

1: Batman
2: Batman Returns
3: Batman & Robin
4: The Dark Knight Rises
5: Batman Begins
6: The Dark Knight
7: A History of Violence
8: Superman
9: Beetlejuice
10: Bedazzled
11: Mars Attacks!
12: The Sentinel
13: Planet of the Apes
14: Man of Steel
15: Suicide Squad
16: The Mask
17: Salton Sea
18: Spider-Man 3
19: The Postman Always Rings Twice
20: Hang 'em High
